In [15]:
import torch
import pandas as pd
import torchvision
import os
from torch.optim import AdamW
from PIL import Image
from torchvision.datasets import ImageFolder

In [16]:
class MappedImageFolder(ImageFolder):
    def __init__(self, root, transform=None, label_mapping=None):
        super().__init__(root, transform=transform)
        self.label_mapping = label_mapping

    def __getitem__(self, index):
        image, label = super().__getitem__(index)
        if self.label_mapping:
            label = self.label_mapping[self.classes[label]]
        return image, label

In [17]:
label_mapping = {str(i): i for i in range(100)} 
t_trans = torchvision.transforms.Compose([
    torchvision.transforms.Resize([256,256]),
    torchvision.transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    torchvision.transforms.RandomHorizontalFlip(0.5),
    torchvision.transforms.RandomRotation(30),
    torchvision.transforms.RandomHorizontalFlip(0.5),
    torchvision.transforms.GaussianBlur(3 , (0.1, 2.0)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

v_trans = torchvision.transforms.Compose([
    torchvision.transforms.Resize([224,224]),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

data = MappedImageFolder('train/', t_trans, label_mapping)

tsize = int(0.8 * len(data))
vsize = int(len(data) - tsize)

train_set , val_set = torch.utils.data.dataset.random_split(data, [tsize,vsize])
val_set.dataset.transform = v_trans

train_loader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=32, shuffle=False)




In [18]:
model = torchvision.models.resnet50(pretrained=True)

count = 0
for name, param in model.named_parameters():
    if count < 6:
        param.requires_grad = False  
    else:
        param.requires_grad = True  
    if "conv" in name or "bn" in name:  
        count += 1
num_ftrs = model.fc.in_features
model.fc = torch.nn.Sequential(
    torch.nn.Dropout(0.6),  
    torch.nn.Linear(num_ftrs, 100) 
)


for param in model.fc.parameters():
    param.requires_grad = True  


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [19]:
#hyperparams
lr = 0.0001
epochs = 10
w = 0.00001

In [20]:
#Criterion and Optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr = lr, weight_decay = w)


In [21]:
#training loop
class Trainer:
    def __init__(self, model, optimizer, criterion, device='cuda'):
        self.model = model.to(device)
        self.optimizer = optimizer
        self.criterion = criterion
        self.device = device

    def evaluate(self, val_loader):
        self.model.eval()
        total_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)
                total_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        loss_val = total_loss / len(val_loader)
        acc_val = correct / total

        return loss_val, acc_val

    def train(self, train_loader, val_loader, epochs=10):
        losses_train = []
        losses_val = []
        accs_train = []
        accs_val = []

        for epoch in range(epochs):
            self.model.train()
            running_loss = 0.0
            correct_train = 0
            total_train = 0

            for inputs, labels in train_loader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)

                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)

                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()

                _, predicted = torch.max(outputs.data, 1)
                total_train += labels.size(0)
                correct_train += (predicted == labels).sum().item()

                running_loss += loss.item()

            loss_train = running_loss / len(train_loader)
            acc_train = correct_train / total_train
            losses_train.append(loss_train)
            accs_train.append(acc_train)
            loss_val, acc_val = self.evaluate(val_loader)
            losses_val.append(loss_val)
            accs_val.append(acc_val)


            print(f"Epoch [{epoch + 1}/{epochs}], "
                  f"Training Loss: {loss_train:.4f}, Training Accuracy: {acc_train:.4f}, "
                  f"Validation Loss: {loss_val:.4f}, Validation Accuracy: {acc_val:.4f}")

        return losses_train, losses_val, accs_train, accs_val

In [22]:
trainer = Trainer(model, optimizer, criterion, device)
losses_train, losses_val, accs_train, accs_val = trainer.train(train_loader, val_loader, epochs)

Epoch [1/10], Training Loss: 4.5208, Training Accuracy: 0.0362, Validation Loss: 4.1550, Validation Accuracy: 0.0900
Epoch [2/10], Training Loss: 3.5028, Training Accuracy: 0.2375, Validation Loss: 3.6278, Validation Accuracy: 0.1350
Epoch [3/10], Training Loss: 2.7763, Training Accuracy: 0.4875, Validation Loss: 3.3036, Validation Accuracy: 0.2350
Epoch [4/10], Training Loss: 2.0727, Training Accuracy: 0.6887, Validation Loss: 2.9501, Validation Accuracy: 0.3250
Epoch [5/10], Training Loss: 1.4901, Training Accuracy: 0.8475, Validation Loss: 2.6958, Validation Accuracy: 0.3850
Epoch [6/10], Training Loss: 1.0202, Training Accuracy: 0.9375, Validation Loss: 2.4977, Validation Accuracy: 0.4450
Epoch [7/10], Training Loss: 0.6537, Training Accuracy: 0.9762, Validation Loss: 2.2772, Validation Accuracy: 0.4650
Epoch [8/10], Training Loss: 0.3902, Training Accuracy: 0.9912, Validation Loss: 2.1123, Validation Accuracy: 0.5250
Epoch [9/10], Training Loss: 0.2374, Training Accuracy: 0.9988, 

In [23]:
test_trans = torchvision.transforms.Compose([
    torchvision.transforms.Resize([224,224]),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])
test_images = sorted([f for f in os.listdir('test/') if f.endswith('.jpg')], key=lambda x: int(x.split('.')[0]))

test_data = []
for img_name in test_images:
    img_path = os.path.join('test/', img_name)
    image = Image.open(img_path).convert('RGB')  
    image = test_trans(image)  
    test_data.append((image, img_name))  

test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False)

In [24]:

def predict(test_loader, label_mapping):
    model.eval() 
    predictions = []
    file_names = []
    with torch.no_grad(): 
        for inputs, names in test_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            predictions.extend(predicted.cpu().numpy())
            file_names.extend(names)
    predictions = [label_mapping[str(pred)] for pred in predictions]
    return predictions, file_names


predictions, file_names = predict(test_loader, label_mapping)

submission = pd.DataFrame({
    'ID': [int(name.split('.')[0]) for name in file_names],
    'Label': predictions
})

submission = submission.sort_values(by='ID')
submission['ID'] = submission['ID'].astype(str) + '.jpg'
submission.to_csv('finalPred.csv', index=False)



In [26]:
torch.save(model.state_dict(), 'final_weights.pth')